In [14]:
# RL deconvolution function adapted from skimage.restoration.richardson_lucy
# https://scikit-image.org/docs/stable/auto_examples/filters/plot_deconvolution.html
# accessed on 12.06.2023

import numpy as np
import numpy.random as npr
from scipy.signal import convolve
from astropy.io import fits
from PIL import Image
from astropy.utils.data import get_pkg_data_filename
import matplotlib.pyplot as plt
from skimage import restoration
from astropy.convolution import Gaussian2DKernel
import torch 
import glob
import pandas as pd
import os

import sys
sys.path.append("../")  # locally defined
from utils_custom import calc_psnr, calc_ssim, calc_sssim # modify utils to utils_custom to avoid conflict with Python package 'utils'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

### THE ONLY CELL BELOW: MODIFY VALUES TO CATER

In [15]:
image_dataset_name = 'FITS_TEST'
image_format = 'fits'
change='4_001'
metric='ssim' # choose the metric of evaluating similarity between hr and lr here, 'ssim' or 'psnr' or 'sssim'
psf=np.array(Gaussian2DKernel(x_stddev=4)) # choose the psf stddev for RL method, '10' or '4' the same as the change from hr to lr

In [16]:
se_hr_dir = '../../Prep_Images/'+image_dataset_name+'_se_hr_'+change # input: selective HR and LR images from preparation step
se_lr_dir = '../../Prep_Images/'+image_dataset_name+'_se_lr_'+change 
rl_dir = '../../RL_Images/'+image_dataset_name+'_'+metric+'_'+change # output: RL deconvolution images

In [17]:
result_data_folder_name = 'result_data_'+image_dataset_name+'/'
result_plot_folder_name = 'result_plot_'+image_dataset_name+'/'

In [18]:
"""
Richardson-Lucy deconvolution function
"""

def richardson_lucy(image_lr, image_hr, metric='sssim', psf=np.array(Gaussian2DKernel(x_stddev=10)), iterations=50, clip=True, filter_epsilon=None):
    """
    Parameters
    ----------
    image : ndarray
       Input degraded image (can be N dimensional).
    psf : ndarray
       The point spread function.
    iterations : int, optional
       Number of iterations. This parameter plays the role of
       regularisation.
    clip : boolean, optional
       True by default. If true, pixel value of the result above 1 or
       under -1 are thresholded for skimage pipeline compatibility.
    filter_epsilon: float, optional
       Value below which intermediate results become 0 to avoid division
       by small numbers.

    References
    ----------
    .. [1] https://en.wikipedia.org/wiki/Richardson%E2%80%93Lucy_deconvolution
    """
   
    metric_map = {
        "sssim": calc_sssim,
        "ssim": calc_ssim,
        "psnr": calc_psnr
    }

    if metric not in metric_map:
        raise ValueError(f"Unsupported metric '{metric}'. Choose from: {list(metric_map.keys())}")

    # metric_fn = metric_map[metric]


    float_type = np.promote_types(image_lr.dtype, np.float32)
    image_lr = image_lr.astype(float_type, copy=False)
    psf = psf.astype(float_type, copy=False)
    im_deconv = np.full(image_lr.shape, 0.5, dtype=float_type)
    psf_mirror = np.flip(psf)
    np.random.seed(123)

    best_score = 0
    for i in range(iterations):
        conv = convolve(im_deconv, psf, mode='same')
        if filter_epsilon:
            relative_blur = np.where(conv < filter_epsilon, 0, image_lr / conv)
        else:
            relative_blur = image_lr / conv

        im_deconv *= convolve(relative_blur, psf_mirror, mode='same')

        if metric == 'sssim':
            res_metric = calc_sssim(img1=torch.Tensor(image_hr.reshape(1, image_hr.shape[0], image_hr.shape[1])), img2=torch.Tensor(im_deconv.reshape(1, im_deconv.shape[0], im_deconv.shape[1])))
        if metric == 'ssim':
            res_metric = calc_ssim(img1=torch.Tensor(image_hr.reshape(1, image_hr.shape[0], image_hr.shape[1])), img2=torch.Tensor(im_deconv.reshape(1, im_deconv.shape[0], im_deconv.shape[1])))
        if metric == 'psnr':
            res_metric = calc_psnr(img1=torch.Tensor(image_hr), img2=torch.Tensor(im_deconv))
        

        if res_metric > best_score:
            best_score = res_metric
            epoch_no = i
            im_deconv_best = im_deconv.copy()


    if clip:
        im_deconv_best[im_deconv_best > 1] = 1
        im_deconv_best[im_deconv_best < -1] = -1  

    return best_score, epoch_no, im_deconv_best

In [19]:
"""
GET RICHARD-LUCY IMAGES FROM LR IMAGES IN ALL FORMATS
"""

np.random.seed(123)
ind = 0
metric_list = []

os.makedirs(rl_dir, exist_ok=True)

for filename in glob.glob(os.path.join(se_hr_dir, '*.'+image_format)):
    ind += 1

    if image_format == 'fits':
        img_hr = get_pkg_data_filename(filename)
        imarray_hr  = fits.getdata(img_hr, ext=0).astype(np.float32)
    else: # tif
        img_hr = Image.open(filename)
        imarray_hr = np.array(img_hr).astype(np.float32)

    filename_lr = os.path.join(se_lr_dir, 'lr_' + filename.split("/")[-1])
    if image_format == 'fits':
        img_lr = get_pkg_data_filename(filename_lr)
        imarray_lr = fits.getdata(img_lr, ext=0).astype(np.float32)
    else: # tif
        img_lr = Image.open(filename_lr)
        imarray_lr = np.array(img_lr).astype(np.float32)


    filename_rl = os.path.join(rl_dir, 'rl_' + filename.split("/")[-1])

    best_score, epoch_no, im_deconv_best = richardson_lucy(image_lr=imarray_lr,image_hr=imarray_hr, metric=metric, psf=psf)

    if image_format == 'fits':
        hdu = fits.PrimaryHDU(im_deconv_best)
        hdu.writeto(filename_rl, overwrite=True)
    else: # tif
        Image.fromarray(im_deconv_best).save(filename_rl)

    # Append results to the list
    metric_list.append({
        "file_index": ind,
        "best_score": float(best_score.item()),
        "epoch_no": epoch_no
    })

    print(ind)

os.makedirs(result_data_folder_name, exist_ok=True)
rl_df = pd.DataFrame(metric_list)
rl_df.to_csv(result_data_folder_name+metric+'_'+change+'.csv')

print(rl_df['best_score'].mean())
print(rl_df['epoch_no'].mean())

1
2
3
4
5
6
7
8
9
10
0.7188503444194794
34.2
